## The Vascular Modeling Toolkit

VMTK 2.0 provides seamless integration of vmtk pypes within the interactive Jupyter Notebook interpreter. By using the new `vmtk.run()` interface, the power of pypes has been extended, allowing for easy access and specification to vmtk data objects within an interactive python session. 

For those familiar with the pypes interface, there are no backwards incompatible API changes. 

To begin to use the new interface, simply `import vmtk`

In [1]:
import vmtk

We can specify a pype by setting an argument string into the vmtk.run interface as normal. The notebook will display all the normal logging data, and any visualizations will appear in a seperate render window

In [2]:
imagePype = vmtk.run('vmtkimagereader -ifile ./aorta.mha --pipe vmtkimageviewer')


Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./aorta.mha
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./aorta.mha
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 0, 0]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Spacing 0.878906 0.878906 1.500090
Origin 156.445000 24.609400 0.000000
Dimensions 157 393 34
Done executing vmtkimagereader.
Output vmtkimagereader members:
    Id = 0
    Image = vtkImageData
    RasToIjkMatrixCoefficients = [1.137778101412438, -0.0, 0.0, -177.99969507546882, -0.0, 1.137778101412438, -0.0, -28

By setting the name `imagePype` to our vmtk.run method, we now have access to every input/output member for every script specified within the pype.

If we wanted to access to vtkImageData which is used to store our image while it's in memory, we can simply assign it to:

In [3]:
aortaImage = imagePype.vmtkimagereader.OutputMembers.Image

In [4]:
aortaImage

(vtkCommonDataModelPython.vtkImageData)0x1217be348

likewise, we can access properties of the render window (even after it has closed out of processing)

In [5]:
windowLevel = imagePype.vmtkimageviewer.InputMembers.WindowLevel

In [6]:
windowLevel

[0.0, 0.0]

we can view the image in a new pype with the new vmtk.run interface. 

Just specify a variable name within `{curly braces}` of the input string, and the object will be seamlessly placed as in input into the new pype instance.

In [11]:
secondImageViewer = vmtk.run('vmtkimagevolumeviewer -i {aortaImage} -preset CT-MIP')


Automatic piping vmtkimagevolumeviewer
Parsing options vmtkimagevolumeviewer
    Image = vtkImageData
    Preset = CT-MIP
Explicit piping vmtkimagevolumeviewer
Input vmtkimagevolumeviewer members:
    Id = 0
    Disabled = 0
    Image = vtkImageData
    ImageInputFileName = 
    ArrayName = 
    vmtkRenderer = None
    Display = 1
    Preset = CT-MIP
    VolumeRenderingMethod = default
    BoxOutline = 0
    ImageOutputFileName = 
Executing vmtkimagevolumeviewer ...
Quit renderer
Done executing vmtkimagevolumeviewer.
Output vmtkimagevolumeviewer members:
    Id = 0
    Image = vtkImageData
    ColorTransferFunction = vtkColorTransferFunction
    GradientOpacityTransferFunction = vtkPiecewiseFunction
    OpacityTransferFunction = vtkPiecewiseFunction
    InterpolationType = 1
    Shade = 1
    SpecularPower = 10.0
    Specular = 0.2
    Diffuse = 0.9
    Ambient = 0.1


In [2]:
fileName = './aorta.mha'
flip = [0, 1, 1]
level = 700

In [4]:
a = vmtk.run('vmtkimagereader -ifile {fileName} -flip {flip} --pipe vmtkmarchingcubes -l {level} --pipe vmtksurfaceviewer')


Automatic piping vmtkimagereader
Parsing options vmtkimagereader
    InputFileName = ./aorta.mha
    Flip = [0, 1, 1]
Explicit piping vmtkimagereader
Input vmtkimagereader members:
    Id = 0
    Disabled = 0
    Format = 
    GuessFormat = 1
    UseITKIO = 1
    Image = 0
    InputFileName = ./aorta.mha
    InputFilePrefix = 
    InputFilePattern = 
    DataExtent = [-1, -1, -1, -1, -1, -1]
    HeaderSize = 0
    DataSpacing = [1.0, 1.0, 1.0]
    DataOrigin = [0.0, 0.0, 0.0]
    DesiredOrientation = native
    DataByteOrder = littleendian
    DataScalarType = float
    FileDimensionality = 3
    Flip = [0, 1, 1]
    AutoOrientDICOMImage = 1
    ImageOutputFileName = 
Executing vmtkimagereader ...
Spacing 0.878906 0.878906 1.500090
Origin 156.445000 24.609400 0.000000
Dimensions 157 393 34
Done executing vmtkimagereader.
Output vmtkimagereader members:
    Id = 0
    Image = vtkImageData
    RasToIjkMatrixCoefficients = [1.137778101412438, -0.0, 0.0, -177.99969507546882, -0.0, 1.13777

In [4]:
myargs2 = 'vmtkcenterlines -i {a.vmtkmarchingcubes.OutputMembers.Surface} --pipe vmtkcenterlineviewer'

In [5]:
myargs2 = 'vmtkrenderer --pipe vmtksurfaceviewer -i {a.vmtkmarchingcubes.OutputMembers.Surface} --pipe vmtkimageviewer -i {a.vmtkimagereader.OutputMembers.Image}'

In [6]:
b = vmtk.run(myargs2)


Automatic piping vmtkrenderer
Parsing options vmtkrenderer
Explicit piping vmtkrenderer
Input vmtkrenderer members:
    Id = 0
    Disabled = 0
    WindowSize = [1200, 900]
    WindowPosition = [50, 50]
    PointSmoothing = 1
    LineSmoothing = 1
    PolygonSmoothing = 0
    Annotations = 1
    Background = [0.1, 0.1, 0.2]
    ScreenshotMagnification = 4
Executing vmtkrenderer ...
Done executing vmtkrenderer.
Output vmtkrenderer members:
    Id = 0
    vmtkRenderer = vmtkRenderer

Automatic piping vmtksurfaceviewer
    vmtkRenderer = vmtkrenderer-0.vmtkRenderer
Parsing options vmtksurfaceviewer
    Surface = vtkPolyData
Explicit piping vmtksurfaceviewer
Input vmtksurfaceviewer members:
    Id = 0
    Disabled = 0
    Surface = vtkPolyData
    SurfaceInputFileName = 
    vmtkRenderer = vmtkRenderer
    Display = 1
    Representation = surface
    Opacity = 1.0
    ArrayName = 
    ScalarRange = [0.0, 0.0]
    ColorMap = cooltowarm
    NumberOfColors = 256
    Legend = 0
    FlatInterp

In [6]:
b.vmtkrenderer.OutputMembers.vmtkRenderer.WindowPosition

[50, 50]

In [6]:
x = 5

In [ ]:
b.vmtkcenterlines.OutputMembers.